# Decision Trees

In [2]:
import numpy as np
from collections import Counter

In [ ]:
def entropy(y):
    # find no. of occurences for each unique y-value
    hist = np.bincount(y)
    # calculate the probability of each class
    ps = hist / len(y)
    # return negative sum of products of each probability that is greater than 0 and the probability's log
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

class Node:
    # initiate class and attributes
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None
    
class DecisionTree:
    # initiate class and attributes
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        # grow tree
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X,y)
        
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (depth >= self.max_depth
            or n_labels == 1
            or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)


    def predict(self, X):
        # traverse tree
        pass

    def _most_common_label(self, y):
        counter = Counter(y)
        pass
